In [2]:
import os
import pandas as pd

import numpy as np
import scanpy as sc
import mudata as md

In [3]:
mdata = md.read_h5mu('../../results/preprocessing_data/multiome_train.h5mu')

In [4]:
mdata

MuData object with n_obs × n_vars = 25551 × 158205
  var:	'gene_id', 'feature_type', 'genome', 'interval'
  2 modalities
    rna:	25551 x 22787
      obs:	'cell_type', 'donor_id'
      var:	'gene_id', 'feature_type', 'genome', 'interval'
      layers:	'counts', 'normalized_count'
    atac:	25551 x 135418
      obs:	'cell_type', 'donor_id'
      var:	'gene_id', 'feature_type', 'genome', 'interval'
      layers:	'counts', 'normalized_count'

In [14]:
def get_pesudo_bulk(adata, by):
    grouped = adata.obs.groupby(by)
    
    df_bulk = pd.DataFrame(
        np.zeros((adata.shape[1], len(grouped)), dtype=np.float32),
        columns=list(grouped.groups.keys()),
        index=adata.var_names
    )
    
    for group, idx in grouped.indices.items():
        X = adata.layers['counts'][idx]
        df_bulk[group] = np.ravel(X.mean(axis=0, dtype=np.float32))
    
    return df_bulk.transpose()

In [15]:
adata_rna = mdata['rna']

In [23]:
adata_rna.var_names

Index(['A1BG', 'A1BG-AS1', 'A2M', 'A2M-AS1', 'A2ML1', 'A2ML1-AS1', 'A3GALT2',
       'A4GALT', 'AAAS', 'AACS',
       ...
       'ZW10', 'ZWILCH', 'ZWINT', 'ZXDA', 'ZXDB', 'ZXDC', 'ZYG11A', 'ZYG11B',
       'ZYX', 'ZZEF1'],
      dtype='object', name='location', length=22787)

In [16]:
df_bulk = get_pesudo_bulk(adata=adata_rna, by=['cell_type', 'donor_id'])

/tmp/ipykernel_71363/1882208799.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = adata.obs.groupby(by)


In [17]:
df_bulk.head()

location,A1BG,A1BG-AS1,A2M,A2M-AS1,A2ML1,A2ML1-AS1,A3GALT2,A4GALT,AAAS,AACS,...,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1
"(B cells, donor_0)",0.042955,0.074456,0.013173,0.001718,0.000573,0.011455,0.000000,0.001145,0.028637,0.027491,...,0.031501,0.021191,0.001718,0.004582,0.005155,0.250860,0.003436,0.110539,0.052119,0.187859
"(B cells, donor_1)",0.080740,0.070648,0.064760,0.009251,0.005046,0.016821,0.001682,0.000841,0.037006,0.046257,...,0.064760,0.038688,0.003364,0.004205,0.023549,0.432296,0.016821,0.210260,0.130362,0.352397
"(B cells, donor_2)",0.037757,0.049199,0.035469,0.016018,0.000000,0.010297,0.000000,0.000000,0.021739,0.035469,...,0.034325,0.022883,0.001144,0.000000,0.004577,0.240275,0.003432,0.145309,0.077803,0.205950
"(Myeloid cells, donor_0)",0.091756,0.068100,0.271685,0.002867,0.007885,0.007885,0.006452,0.001434,0.075986,0.015771,...,0.076702,0.029391,0.005018,0.002151,0.006452,0.370609,0.002867,0.236559,0.561290,0.741219
"(Myeloid cells, donor_1)",0.136185,0.077708,0.390111,0.012559,0.015699,0.012559,0.003532,0.038462,0.081240,0.035714,...,0.195448,0.071429,0.006279,0.006672,0.010597,0.619701,0.000392,0.463110,0.626763,1.187585


In [18]:
# filter genes by training data
df_de_train = pd.read_parquet('../../data/de_train.parquet')
sel_genes = df_de_train.columns.values.tolist()[5:]

In [24]:
df_de_train.head()

,cell_type,sm_name,sm_lincs_id,SMILES,control,A1BG,A1BG-AS1,A2M,A2M-AS1,A2MP1,...,ZUP1,ZW10,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1
0,NK cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.104720,-0.077524,-1.625596,-0.144545,0.143555,...,-0.227781,-0.010752,-0.023881,0.674536,-0.453068,0.005164,-0.094959,0.034127,0.221377,0.368755
1,T cells CD4+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.915953,-0.884380,0.371834,-0.081677,-0.498266,...,-0.494985,-0.303419,0.304955,-0.333905,-0.315516,-0.369626,-0.095079,0.704780,1.096702,-0.869887
2,T cells CD8+,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,-0.387721,-0.305378,0.567777,0.303895,-0.022653,...,-0.119422,-0.033608,-0.153123,0.183597,-0.555678,-1.494789,-0.213550,0.415768,0.078439,-0.259365
3,T regulatory cells,Clotrimazole,LSM-5341,Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1,False,0.232893,0.129029,0.336897,0.486946,0.767661,...,0.451679,0.704643,0.015468,-0.103868,0.865027,0.189114,0.224700,-0.048233,0.216139,-0.085024
4,NK cells,Mometasone Furoate,LSM-3349,C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...,False,4.290652,-0.063864,-0.017443,-0.541154,0.570982,...,0.758474,0.510762,0.607401,-0.123059,0.214366,0.487838,-0.819775,0.112365,-0.122193,0.676629


In [19]:
sel_genes[:10]

['A1BG',
 'A1BG-AS1',
 'A2M',
 'A2M-AS1',
 'A2MP1',
 'A4GALT',
 'AAAS',
 'AACS',
 'AAGAB',
 'AAK1']

In [20]:
len(sel_genes)

18211

In [21]:
df_bulk.shape

(18, 22787)

In [22]:
df_bulk.loc[:, 'A2MP1']

KeyError: 'A2MP1'